# Difference between BM25 similiarity and Semantic similiarity

### Upgrade PyTorch and restart Kernel

In [ ]:
!pip install --upgrade torch

In [ ]:
from IPython.display import display_html
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
restartkernel()

### Verify PyTorch version

In [ ]:
import torch
print(torch.__version__)

### Install required libarary, such as HuggingFace

In [1]:
!pip install -q transformers
!pip install -U sentence-transformers rank_bm25

## Difference between BM25 similiarity and Semantic similiarity
### BM25 similiarities

In [ ]:
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string
from tqdm.autonotebook import tqdm
import numpy as np

passages=["does this work with xbox?",
          "Does the M70 work with Android phones?", 
          "does this work with iphone?",
          "Can this work with an xbox "
         ]

def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)

        if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
            tokenized_doc.append(token)
    return tokenized_doc


tokenized_corpus = []
for passage in tqdm(passages):
    tokenized_corpus.append(bm25_tokenizer(passage))

bm25 = BM25Okapi(tokenized_corpus)

bm25_scores = bm25.get_scores(bm25_tokenizer(passages[0]))

all_sentence_combinations = []
for i in range(len(bm25_scores)):
    all_sentence_combinations.append([bm25_scores[i], i])

all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

print("Top most similar pairs:")
for score, i in all_sentence_combinations[0:4]:
    print("{} \t {} \t {:.4f}".format(passages[i],bm25_tokenizer(passages[i]),bm25_scores[i]))
    


### Semantic Similiarities

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

#Encode all sentences
embeddings = model.encode(passages)

#Compute cosine similarity between all pairs
cos_sim = util.cos_sim(embeddings, embeddings)

#cosine similarity score with query
all_sentence_combinations = []
for i in range(len(cos_sim)):
    all_sentence_combinations.append([cos_sim[0][i], i])

#Sort list by the highest cosine similarity score
all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

print("Top most similar pairs:")
for score, i in all_sentence_combinations[0:4]:
    print("{} \t {:.4f}".format(passages[i],cos_sim[0][i]))